In [ ]:
!git clone https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning.git

In [ ]:
import pandas as pd
import numpy as np



# Encode ZIP file to video

In [8]:
import cv2
import numpy as np
import zipfile

# Open the zip file and read the contents of the first file as bytes
with zipfile.ZipFile('model.zip', 'r') as zip_file:
    file_bytes = zip_file.read(zip_file.namelist()[0])


# Convert the bytes to a binary string
binary_str = ''.join(format(byte, '08b') for byte in file_bytes)

# Compute the number of pixels needed to encode the binary string
pixel_count = len(binary_str) // 3 + 1

# Create a black image with the desired dimensions
img = np.zeros((1080, 1920, 3), dtype=np.uint8)

# Encode the binary string as pixels in the image
for i in range(pixel_count):
    start = i * 3
    end = start + 3
    bits = binary_str[start:end].ljust(3, '0')
    pixel = tuple(int(bits[j:j+1]) * 255 for j in range(3))
    x, y = divmod(i, 1920)
    img[x, y] = pixel

# Save the image as a video with a low frame rate
fps = 30
frame_count = fps*8
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('file.mp4', fourcc, fps, (1920, 1080))
for i in range(frame_count):
    out.write(img)
out.release()
cv2.destroyAllWindows()


IndexError: ignored

In [ ]:
# Decode from Mp4

In [6]:
import cv2
import numpy as np
import zipfile

# Open the video file and extract the pixel values from each frame
cap = cv2.VideoCapture('file.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pixels = []
for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    pixels.extend(frame.reshape(-1, 3))

# Convert the pixel values to a binary string
binary_str = ''.join(str(pixel[0] // 255) for pixel in pixels)

# Convert the binary string to bytes
file_bytes = bytearray(int(binary_str[i:i+8], 2) for i in range(0, len(binary_str), 8))

# Write the bytes to a new zip file
with zipfile.ZipFile('decoded.zip', 'w') as zip_file:
    with zipfile.ZipFile('file.zip', 'r') as orig_zip_file:
        for info in orig_zip_file.infolist():
            with orig_zip_file.open(info) as orig_file:
                if info.filename != 'file.txt':
                    zip_file.writestr(info, orig_file.read())
                else:
                    zip_file.writestr(info.filename, file_bytes)
